#### Creating old new column in both Train & Test data sets

- The strategy is to create a new feature column in the dataset named old_new 
- So how do we decide old or new ?
- we will consider all the first day medicines as old medicines
- It will be stored in old_med_dict(dictonary of dictonaries for faster access)
- Then if a new medicine is introduced we call it as new medicine only on that first day and update the above dictonary
- we check each row of medicine for every city whether it is present in the above old_med_dict
- if yes we update it as old
- if no we update as new and update that city medicine combination to old_med_dict

In [1]:
import numpy as np
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
path=os.getcwd()
path='C:\\Users\\jayam\\Downloads\\7879phd\\model_building_part_2\\'

In [26]:
test=pd.read_csv("test_finaldata.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])# reading test data

In [4]:
test.head()

,id,year,month,day,city,medicine,date,discounted,footfall,year_month,city_medicine,week
0,1,2018,7,1,1,1292,2018-07-01,0,NaN,2018_07,1_1292,26
1,2,2018,7,1,1,1,2018-07-01,0,NaN,2018_07,1_1,26
2,3,2018,7,1,1,2,2018-07-01,1,NaN,2018_07,1_2,26
3,4,2018,7,1,1,3,2018-07-01,0,NaN,2018_07,1_3,26
4,5,2018,7,1,1,4,2018-07-01,0,NaN,2018_07,1_4,26


In [21]:
train=pd.read_csv("train_footfalldata.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])# reading train data

In [6]:
train.head()

,year,month,day,city,medicine,date,sales,discounted,year_month,city_medicine,week,footfall_perday_permed
0,2015,1,2,1,1,2015-01-02,24.0,0,2015_01,1_1,1,0.385804
1,2015,1,2,1,2,2015-01-02,144.0,0,2015_01,1_2,1,2.314824
2,2015,1,2,1,3,2015-01-02,84.0,0,2015_01,1_3,1,1.350314
3,2015,1,2,1,4,2015-01-02,24.0,0,2015_01,1_4,1,0.385804
4,2015,1,2,1,5,2015-01-02,8.0,0,2015_01,1_5,1,0.128601


In [27]:
for col in ["year","month","day","city","medicine","week"]:
    train[col]=train[col].astype(int)

In [8]:
train.dtypes# check dtypes

year                        int32
month                       int32
day                         int32
city                        int32
medicine                    int32
date                       object
sales                     float64
discounted                  int64
year_month                 object
city_medicine              object
week                        int32
footfall_perday_permed    float64
dtype: object

In [28]:
test.dtypes# check dtypes

id                 int64
year               int64
month              int64
day                int64
city               int64
medicine           int64
date              object
discounted         int64
footfall         float64
year_month        object
city_medicine     object
week               int64
dtype: object

In [29]:
test.rename(columns={"footfall":"footfall_perday_permed"},inplace=True)

In [6]:
def old_med_dict(df,date,cities):# function  for storing the first  day medicine list
    mydict={} # creating dictonary
    for c in cities:# on the first day we are taking medicines for each city and storing it in temp
        temp = list(df[(df["date"]==date)&(df["city"]==c)]["medicine"].value_counts().index)
        mydict[c]= dict(zip(temp, temp))# creating dictonary of dictonaries for each city as key 
    return mydict      # and value contains another dictonary having key and value as medicines

In [ ]:
trn_dict=old_med_dict(train,"2015-01-02",list(range(1,11)))# calling the above function

In [ ]:
#create old_new column
#for every row in the df:
    #check for medicine and city in old_meds_dictionary:
        #if it is present then:
            #update "old" in the "old_new" column
        #else:
            #update "new" in the "old_new" column
            #add the medicine in the city dictionary of the "old_meds_dictionary"
#done

In [5]:
#Update Dictionary 
def check_medicine_city_in_old_meds(medicine, city, old_meds):# checking function if medicine is there or not in old_med_dict
    if city in old_meds.keys():# if only city is present
        city_dict = old_meds[city]#we are going inside the dictonary of the passed city and storing dictonary of medicnes 
        if medicine in city_dict.keys():# if medicine is present in dictonary return true else false
            return True
    return False
def create_old_new_column(df, old_meds_dict):# creating a column old or new depending on medicine 
    df["old_new"] = "old"# intially store everyting as old
    for index, row in df.iterrows():# iterate on each row
        if False == check_medicine_city_in_old_meds(row["medicine"], row["city"], old_meds_dict):# check whether it is present in old med dict
            df.loc[df.index == index,"old_new"] = "new"# we are updating the column as new as it is not in old_med_dict 
            old_meds_dict[row["city"]][row["medicine"]] = row["medicine"]# updating old_med_dict

In [12]:
train["type"]="train"
test["type"]="test"

#### Creating separate files as the system has crashed last time for memory optimization as well during run time

In [18]:
merge_train=train[["city","medicine","date","type"]]

In [20]:
merge_test=test[["city","medicine","date","type"]]

In [23]:
merge_train=pd.concat([merge_train,merge_test])# merging train and test with only required columns

In [25]:
merge_train.to_csv("merged_exp.csv",index=False)# saving it in csv file to avoid system crashes

In [3]:
file=pd.read_csv("merged_exp.csv", sep=",", na_values=["?",",","#","NaN","unknown",""])# reading the merged file

In [7]:
file_dict=old_med_dict(file,"2015-01-02",list(range(1,11)))# creating the intial dictonary for old new medicines

In [10]:
create_old_new_column(file,file_dict)# running the fuction with required columns

In [44]:
file.to_csv("file.csv",index=False)# save the file in case of reference

In [12]:
file["old_new"].value_counts()

old    23354243
new       21033
Name: old_new, dtype: int64

In [66]:
test1=test1.reset_index()

In [16]:
file[file["type"]=="test"]["old_new"].value_counts()

old    773222
new       313
Name: old_new, dtype: int64

In [19]:
train1=file[file["type"]=="train"]
test1=file[file["type"]=="test"]

In [23]:
train["old_new"]=train1["old_new"]# adding old new column to train data

In [41]:
test["old_new"]=test1["old_new"]# adding old new column to test data

In [69]:
train.to_csv("train_final.csv",index=false)# save train file

In [68]:
test.to_csv("test_final.csv",index=false)# save test file